[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hw-oh/wandb_e2e_demo/blob/main/models/image_detection/image_detection.ipynb)

In [ ]:
!pip install -q wandb ultralytics wandb-workspaces

In [ ]:
import wandb
import os
from google.colab import userdata

WANDB_API_KEY = userdata.get("WANDB_API_KEY")
WANDB_ENTITY = userdata.get("WANDB_ENTITY")
WANDB_PROJECT = userdata.get("WANDB_PROJECT")
WANDB_REGISTRY_NAME = userdata.get("WANDB_REGISTRY_NAME")

os.environ["WANDB_API_KEY"] = WANDB_API_KEY

wandb.login(key=WANDB_API_KEY)
print(f"Entity: {WANDB_ENTITY}")
print(f"Project: {WANDB_PROJECT}")

# Object Detection Demo — W&B 전체 기능 체험

## 개요

이 노트북은 **COCO128** 데이터셋과 **YOLOv8n (nano)** 모델을 사용하여
객체 탐지(Object Detection)를 수행하면서, W&B의 핵심 기능을 전부 체험합니다.

## 다루는 W&B 기능

| 기능 | 설명 |
|------|------|
| **Experiment Tracking** | 학습 메트릭 실시간 추적 (`wandb.init`, `wandb.log`, `wandb.config`) |
| **Media Logging** | BBox 오버레이 시각화 (`wandb.Image` + `boxes` 파라미터) |
| **Tables** | 데이터셋 미리보기 및 예측 결과 비교 (`wandb.Table`) |
| **Artifacts** | 데이터셋/모델 버저닝 및 계보(lineage) 추적 |
| **Model Registry** | 모델 등록 및 alias 관리 (staging) |
| **Sweeps** | 베이지안 하이퍼파라미터 최적화 |
| **Reports** | 프로그래밍 방식 실험 리포트 생성 |

## 데이터셋
- **COCO128**: COCO 데이터셋의 소형 서브셋 (128장, 80 classes)
- Ultralytics 내장 — `model.train()` 시 자동 다운로드
- 빠른 데모에 최적

## 모델
- **YOLOv8n** (nano) — Ultralytics pretrained on COCO
- ~3.2M 파라미터, ~6MB 모델 파일
- Colab T4에서 빠른 학습, Streamlit Cloud(CPU)에서 빠른 추론

In [ ]:
from ultralytics import YOLO
from ultralytics.data.utils import check_det_dataset
from PIL import Image
import numpy as np
import random
import glob
import yaml
import time

CONFIG = {
    "model_name": "yolov8n",
    "dataset": "coco128",
    "epochs": 30,
    "imgsz": 640,
    "lr0": 0.01,
    "batch": 16,
    "num_classes": 80,
}

COCO_CLASSES = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck",
    "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
    "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra",
    "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee",
    "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
    "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
    "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange",
    "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch",
    "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse",
    "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear",
    "hair drier", "toothbrush",
]

CLASS_LABELS = {i: name for i, name in enumerate(COCO_CLASSES)}

print(f"COCO classes: {len(COCO_CLASSES)}개")

In [ ]:
# === COCO128 데이터셋 다운로드 ===

data_info = check_det_dataset("coco128.yaml")
DATASET_DIR = data_info["path"]
print(f"데이터셋 경로: {DATASET_DIR}")

train_images = glob.glob(f"{DATASET_DIR}/images/train2017/*.jpg")
print(f"Train 이미지: {len(train_images)}장")

In [ ]:
# === 데이터셋 Artifact 생성 ===

run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    config=CONFIG,
    job_type="data-versioning",
    name="coco128-data-versioning",
)

artifact = wandb.Artifact(
    "coco128",
    type="dataset",
    description="COCO128 dataset (Ultralytics subset, 128 images, 80 classes)",
    metadata={
        "num_images": len(train_images),
        "num_classes": 80,
        "classes": COCO_CLASSES,
        "format": "YOLO",
        "source": "Ultralytics COCO128",
    },
)
artifact.add_dir(DATASET_DIR)
run.log_artifact(artifact)
print("데이터셋 Artifact 로깅 완료!")

In [ ]:
# === 샘플 이미지 + BBox 시각화 (wandb.Table) ===

def parse_yolo_label(label_path, img_w, img_h):
    """YOLO format 라벨을 wandb box_data로 변환한다."""
    box_data = []
    if not os.path.exists(label_path):
        return box_data
    with open(label_path) as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) < 5:
                continue
            cls_id = int(parts[0])
            x_c, y_c, w, h = float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4])
            box_data.append({
                "position": {"middle": [x_c, y_c], "width": w, "height": h},
                "class_id": cls_id,
                "box_caption": COCO_CLASSES[cls_id] if cls_id < len(COCO_CLASSES) else str(cls_id),
            })
    return box_data


table = wandb.Table(columns=["Image with BBox", "Image", "Num Objects", "Classes"])

sample_images = random.sample(train_images, min(20, len(train_images)))

for img_path in sample_images:
    img = Image.open(img_path)
    img_w, img_h = img.size

    label_path = img_path.replace("/images/", "/labels/").replace(".jpg", ".txt")
    box_data = parse_yolo_label(label_path, img_w, img_h)

    img_with_boxes = wandb.Image(
        img,
        boxes={
            "ground_truth": {
                "box_data": box_data,
                "class_labels": CLASS_LABELS,
            }
        } if box_data else {},
    )

    class_names = list({bd["box_caption"] for bd in box_data})

    table.add_data(
        img_with_boxes,
        wandb.Image(img),
        len(box_data),
        ", ".join(class_names),
    )

wandb.log({"dataset_preview": table})
wandb.finish()
print("데이터셋 미리보기 테이블 로깅 완료!")

In [ ]:
# === 모델 로드 (YOLOv8n) ===

model = YOLO("yolov8n.pt")
print(model.info())

In [ ]:
# === 학습 전 베이스라인 모델 등록 ===

run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    config=CONFIG,
    job_type="model-baseline",
    name="yolov8n-coco128-pretrained",
)

data_artifact = run.use_artifact(f"{WANDB_ENTITY}/{WANDB_PROJECT}/coco128:latest")

# pretrained 모델을 베이스라인으로 등록
baseline_artifact = wandb.Artifact(
    "yolov8n-coco128",
    type="model",
    description="YOLOv8n pretrained on COCO (baseline, before fine-tuning)",
    metadata={
        "model_type": "detection",
        "model_architecture": "yolov8n",
        "dataset": "coco128",
        "num_classes": 80,
        "classes": COCO_CLASSES,
        "framework": "ultralytics",
        "input_size": [3, 640, 640],
        "trained": False,
    },
)
baseline_artifact.add_file("yolov8n.pt", name="best.pt")
run.log_artifact(baseline_artifact)

run.link_artifact(
    baseline_artifact,
    f"{WANDB_REGISTRY_NAME}/coco128-detector",
    aliases=["baseline"],
)
print("학습 전 모델을 Registry에 'baseline' alias로 등록 완료!")

wandb.finish()

In [ ]:
# === YOLOv8n 학습 + W&B 메트릭 로깅 ===

os.environ["WANDB_DISABLED"] = "true"  # Ultralytics 내장 콜백 비활성화 (수동 로깅 데모)

run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    config=CONFIG,
    job_type="training",
    name="yolov8n-coco128-train",
)

data_artifact = run.use_artifact(f"{WANDB_ENTITY}/{WANDB_PROJECT}/coco128:latest")

model = YOLO("yolov8n.pt")

results = model.train(
    data="coco128.yaml",
    epochs=CONFIG["epochs"],
    imgsz=CONFIG["imgsz"],
    lr0=CONFIG["lr0"],
    batch=CONFIG["batch"],
    project="runs/detect",
    name="coco128_train",
    exist_ok=True,
    verbose=True,
)

# Ultralytics 학습 결과에서 에포크별 CSV 읽어서 W&B에 수동 로깅
import csv

csv_path = os.path.join(results.save_dir, "results.csv")
if os.path.exists(csv_path):
    with open(csv_path) as f:
        reader = csv.DictReader(f)
        for row in reader:
            row = {k.strip(): v.strip() for k, v in row.items()}
            wandb.log({
                "epoch": int(row["epoch"]),
                "train/box_loss": float(row["train/box_loss"]),
                "train/cls_loss": float(row["train/cls_loss"]),
                "train/dfl_loss": float(row["train/dfl_loss"]),
                "val/box_loss": float(row["val/box_loss"]),
                "val/cls_loss": float(row["val/cls_loss"]),
                "val/dfl_loss": float(row["val/dfl_loss"]),
                "val/precision": float(row["metrics/precision(B)"]),
                "val/recall": float(row["metrics/recall(B)"]),
                "val/mAP50": float(row["metrics/mAP50(B)"]),
                "val/mAP50-95": float(row["metrics/mAP50-95(B)"]),
            })

BEST_PT_PATH = os.path.join(results.save_dir, "weights/best.pt")
print(f"Best model: {BEST_PT_PATH}")
print(f"학습 완료! mAP50: {results.results_dict.get('metrics/mAP50(B)', 'N/A')}")

In [ ]:
# === Detection 결과 BBox 시각화 로깅 ===

best_model = YOLO(BEST_PT_PATH)

sample_for_viz = random.sample(train_images, min(16, len(train_images)))

detection_images = []
for img_path in sample_for_viz:
    img = Image.open(img_path)
    img_w, img_h = img.size

    preds = best_model(img_path, verbose=False)
    box_data = []
    for box in preds[0].boxes:
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        box_data.append({
            "position": {
                "minX": x1 / img_w,
                "minY": y1 / img_h,
                "maxX": x2 / img_w,
                "maxY": y2 / img_h,
            },
            "class_id": int(box.cls),
            "scores": {"confidence": float(box.conf)},
            "box_caption": f"{COCO_CLASSES[int(box.cls)]} {float(box.conf):.2f}",
        })

    detection_images.append(
        wandb.Image(
            img,
            boxes={
                "predictions": {
                    "box_data": box_data,
                    "class_labels": CLASS_LABELS,
                }
            } if box_data else {},
        )
    )

wandb.log({"detection_results": detection_images})
print(f"{len(sample_for_viz)}장의 이미지에 대한 detection 결과 로깅 완료!")

In [ ]:
# === 예측 결과 wandb.Table ===

pred_table = wandb.Table(columns=[
    "Image + BBox", "Rendered", "Num Objects", "Detected Classes", "Avg Confidence",
])

eval_images = random.sample(train_images, min(50, len(train_images)))

for img_path in eval_images:
    img = Image.open(img_path)
    img_w, img_h = img.size

    preds = best_model(img_path, verbose=False)
    r = preds[0]

    box_data = []
    detected_classes = set()
    confs = []

    for box in r.boxes:
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        cls_id = int(box.cls)
        conf = float(box.conf)
        box_data.append({
            "position": {
                "minX": x1 / img_w, "minY": y1 / img_h,
                "maxX": x2 / img_w, "maxY": y2 / img_h,
            },
            "class_id": cls_id,
            "scores": {"confidence": conf},
            "box_caption": f"{COCO_CLASSES[cls_id]} {conf:.2f}",
        })
        detected_classes.add(COCO_CLASSES[cls_id])
        confs.append(conf)

    img_with_boxes = wandb.Image(
        img,
        boxes={
            "predictions": {
                "box_data": box_data,
                "class_labels": CLASS_LABELS,
            }
        } if box_data else {},
    )

    rendered = Image.fromarray(r.plot()[..., ::-1])  # BGR -> RGB

    avg_conf = np.mean(confs) if confs else 0.0
    pred_table.add_data(
        img_with_boxes,
        wandb.Image(rendered),
        len(r.boxes),
        ", ".join(sorted(detected_classes)),
        f"{avg_conf:.2%}",
    )

wandb.log({"prediction_table": pred_table})
print(f"예측 결과 테이블 ({len(eval_images)}장) 로깅 완료!")

In [ ]:
# === 모델 Artifact 저장 + Model Registry 등록 ===

final_metrics = results.results_dict

model_artifact = wandb.Artifact(
    "yolov8n-coco128",
    type="model",
    description="YOLOv8n fine-tuned on COCO128",
    metadata={
        "model_type": "detection",
        "model_architecture": "yolov8n",
        "dataset": "coco128",
        "num_classes": 80,
        "classes": COCO_CLASSES,
        "framework": "ultralytics",
        "input_size": [3, 640, 640],
        "trained": True,
        "epochs": CONFIG["epochs"],
        "best_mAP50": final_metrics.get("metrics/mAP50(B)", 0),
        "best_mAP50-95": final_metrics.get("metrics/mAP50-95(B)", 0),
        "best_precision": final_metrics.get("metrics/precision(B)", 0),
        "best_recall": final_metrics.get("metrics/recall(B)", 0),
    },
)
model_artifact.add_file(BEST_PT_PATH, name="best.pt")
run.log_artifact(model_artifact)

run.link_artifact(
    model_artifact,
    f"{WANDB_REGISTRY_NAME}/coco128-detector",
    aliases=["staging"],
)
print("학습된 모델을 Registry에 'staging' alias로 등록 완료!")

wandb.finish()

# 하이퍼파라미터 Sweep

W&B Sweeps를 사용하여 YOLOv8n의 하이퍼파라미터를 베이지안 최적화합니다.

**탐색 파라미터:**
- `lr0`: 초기 학습률 (1e-5 ~ 1e-2, log-uniform)
- `imgsz`: 입력 이미지 크기 (320 / 640)
- `augment`: 데이터 증강 활성화 여부

**최적화 목표:** `val/mAP50` 최대화

In [ ]:
# === Sweep ===

sweep_config = {
    "method": "bayes",
    "metric": {"name": "val/mAP50", "goal": "maximize"},
    "parameters": {
        "lr0": {"min": 1e-5, "max": 1e-2, "distribution": "log_uniform_values"},
        "imgsz": {"values": [320, 640]},
        "augment": {"values": [True, False]},
    },
}


def sweep_train():
    run = wandb.init()
    config = wandb.config

    sweep_model = YOLO("yolov8n.pt")
    sweep_results = sweep_model.train(
        data="coco128.yaml",
        epochs=3,
        imgsz=config.imgsz,
        lr0=config.lr0,
        augment=config.augment,
        batch=CONFIG["batch"],
        project="runs/detect_sweep",
        name=f"sweep_{run.id}",
        exist_ok=True,
        verbose=False,
    )

    metrics = sweep_results.results_dict
    wandb.log({
        "val/mAP50": metrics.get("metrics/mAP50(B)", 0),
        "val/mAP50-95": metrics.get("metrics/mAP50-95(B)", 0),
        "val/precision": metrics.get("metrics/precision(B)", 0),
        "val/recall": metrics.get("metrics/recall(B)", 0),
    })

    wandb.finish()


sweep_id = wandb.sweep(sweep_config, project=WANDB_PROJECT, entity=WANDB_ENTITY)
wandb.agent(sweep_id, function=sweep_train, count=5)
print("Sweep 완료!")

In [ ]:
# === W&B Report 생성 ===

import wandb_workspaces.reports.v2 as wr

report = wr.Report(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    title="COCO128 Object Detection — 실험 결과 리포트",
    description="YOLOv8n COCO128 fine-tuning 실험 결과 및 Sweep 분석",
)

report.blocks = [
    wr.TableOfContents(),

    wr.H1("1. 실험 개요"),
    wr.P(
        "COCO128 데이터셋에 대한 YOLOv8n 객체 탐지 실험 결과를 정리합니다. "
        "W&B의 Experiment Tracking, Artifacts, Sweeps, Model Registry, Media Logging 기능을 활용하였습니다."
    ),

    wr.H1("2. 학습 결과"),
    wr.PanelGrid(
        runsets=[
            wr.Runset(entity=WANDB_ENTITY, project=WANDB_PROJECT)
        ],
        panels=[
            wr.LinePlot(title="Box Loss (Train)", x="epoch", y=["train/box_loss"]),
            wr.LinePlot(title="Box Loss (Val)", x="epoch", y=["val/box_loss"]),
            wr.LinePlot(title="mAP@0.5", x="epoch", y=["val/mAP50"]),
            wr.LinePlot(title="mAP@0.5:0.95", x="epoch", y=["val/mAP50-95"]),
            wr.LinePlot(title="Precision", x="epoch", y=["val/precision"]),
            wr.LinePlot(title="Recall", x="epoch", y=["val/recall"]),
        ],
    ),

    wr.H1("3. Sweep 분석"),
    wr.P("Bayesian 최적화를 통한 하이퍼파라미터 탐색 결과:"),
    wr.PanelGrid(
        runsets=[
            wr.Runset(entity=WANDB_ENTITY, project=WANDB_PROJECT)
        ],
        panels=[
            wr.ParallelCoordinatesPlot(
                columns=[
                    wr.ParallelCoordinatesPlotColumn(metric="c::lr0"),
                    wr.ParallelCoordinatesPlotColumn(metric="c::imgsz"),
                    wr.ParallelCoordinatesPlotColumn(metric="c::augment"),
                    wr.ParallelCoordinatesPlotColumn(metric="val/mAP50"),
                ],
            ),
            wr.ScalarChart(title="Best mAP@0.5", metric="val/mAP50"),
            wr.BarPlot(title="mAP@0.5 by Run", metrics=["val/mAP50"]),
        ],
    ),

    wr.H1("4. 다음 단계"),
    wr.P(
        "최적 모델을 Model Registry의 'production' alias로 승격하여 "
        "배포 파이프라인을 트리거합니다."
    ),
]

report.save()
print(f"Report 생성 완료! URL: {report.url}")

In [ ]:
wandb.finish()
print("모든 W&B 리소스가 정리되었습니다. 데모 완료!")